In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

import os

def make_dir(path):
    if os.path.isdir(data_path):
        pass
    else:
        os.path.make_dir(path)

In [23]:
# 初期設定
import sagemaker
import os
import boto3
import re
import numpy as np

sagemaker_session = sagemaker.Session()

# AWS設定
role = 'FullAccessHan'
region = boto3.Session().region_name
bucket='sagemaker-han'
prefix = 'sagemaker'
testdata_dir = 'batch_transform_json'
bucket_path = 'https://s3-{}.amazonaws.com/{}'.format(region,bucket)

# Local設定
data_dir = 'cifar10/testing'

In [24]:
# Build image list
path = './data/images'
image_prefix = os.path.join(bucket_path, testdata_dir)
img_files = [os.path.join(image_prefix, f) for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]

# # Write jsonl file
json_file = "./data/test_img_path.json"
with open(json_file, mode="w") as f:
    for img_file in img_files:
        json_obj = {"inputs": img_file}
        json.dump(json_obj, f)
        f.write('\n')

# Upload to S3
json_input = sagemaker_session.upload_data(path=json_file, bucket=bucket, key_prefix=testdata_dir)

In [29]:
from sagemaker.pytorch.model import PyTorchModel

model_data = 's3://sagemaker-us-west-2-608095525235/pytorch-training-2022-08-20-08-30-28-845/output/model.tar.gz'

pytorch_model = PyTorchModel(model_data = model_data,
                             entry_point='resnet_deploy.py',
                             source_dir = 'src',
                             framework_version='1.12.0',
                             py_version='py38',
                             role = role)

In [33]:
output_s3_path = 'https://sagemaker-han.s3.us-west-2.amazonaws.com/sagemaker/batch_transform_output_json'
max_concurrent_transforms = 100
max_payload = 1

transformer = pytorch_model.transformer(instance_count=1, 
                                        instance_type="ml.m5.large",
                                        output_path=output_s3_path,
                                        max_concurrent_transforms = max_concurrent_transforms,
                                        max_payload = max_payload
                                       )

In [34]:
json_input = sagemaker_session.upload_data(path=json_file, bucket=bucket, key_prefix=testdata_dir)
transformer.transform(
    data=json_input,
    content_type='application/json',    
    split_type='Line',
    wait=True,
)

2022-08-21T08:15:11,708 [INFO ] main org.pytorch.serve.servingsdk.impl.PluginsManager - Initializing plugins manager...
2022-08-21T08:15:11,790 [INFO ] main org.pytorch.serve.ModelServer - 
Torchserve version: 0.6.0
TS Home: /opt/conda/lib/python3.8/site-packages
Current directory: /
Temp directory: /home/model-server/tmp
Number of GPUs: 0
Number of CPUs: 2
Max heap size: 962 M
Python executable: /opt/conda/bin/python3.8
Config file: /etc/sagemaker-ts.properties
Inference address: http://0.0.0.0:8080
Management address: http://0.0.0.0:8080
Metrics address: http://127.0.0.1:8082
Model Store: /.sagemaker/ts/models
Initial Models: model=/opt/ml/model
Log dir: /logs
Metrics dir: /logs
Netty threads: 0
Netty client threads: 0
Default workers per model: 2
Blacklist Regex: N/A
Maximum Response Size: 6553500
Maximum Request Size: 6553500
2022-08-21T08:15:11,708 [INFO ] main org.pytorch.serve.servingsdk.impl.PluginsManager - Initializing plugins manager...
2022-08-21T08:15:11,790 [INFO ] main o

2022-08-21T08:15:21.190:[sagemaker logs]: sagemaker-han/batch_transform_json/test_img_path.json: Bad HTTP status received from algorithm: 500
2022-08-21T08:15:21.191:[sagemaker logs]: sagemaker-han/batch_transform_json/test_img_path.json: 
2022-08-21T08:15:21.191:[sagemaker logs]: sagemaker-han/batch_transform_json/test_img_path.json: Message:
2022-08-21T08:15:21.191:[sagemaker logs]: sagemaker-han/batch_transform_json/test_img_path.json: 'str' object has no attribute 'inputs'
2022-08-21T08:15:21.191:[sagemaker logs]: sagemaker-han/batch_transform_json/test_img_path.json: Traceback (most recent call last):
2022-08-21T08:15:21.191:[sagemaker logs]: sagemaker-han/batch_transform_json/test_img_path.json:   File "/opt/conda/lib/python3.8/site-packages/sagemaker_inference/transformer.py", line 128, in transform
2022-08-21T08:15:21.191:[sagemaker logs]: sagemaker-han/batch_transform_json/test_img_path.json:     result = self._transform_fn(self._model, input_data, content_type, accept)
2022-0

UnexpectedStatusException: Error for Transform job pytorch-inference-2022-08-21-08-10-14-217: Failed. Reason: AlgorithmError: See job logs for more information

In [ ]:
transformer.stop_transform_job()